<a href="https://colab.research.google.com/github/Adeelzafar/NLP-Course/blob/main/NLP_Deep_Dive_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install fastai
!pip install sentencepiece!=0.1.90

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:00


In [3]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [4]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [5]:
txt = files[0].open().read(); txt[:75]

'This is one of the worst anime series I have ever seen. When I watched the '

In [6]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#238) ['This','is','one','of','the','worst','anime','series','I','have','ever','seen','.','When','I','watched','the','manga','review','in','a','magazine',',','I','thought','it','was','maybe','interesting',','...]


In [7]:
txts = L(o.open().read() for o in files[:2000])

In [8]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [9]:
subword(1000)

'▁This ▁is ▁one ▁of ▁the ▁worst ▁an im e ▁series ▁I ▁have ▁ever ▁seen . ▁When ▁I ▁watched ▁the ▁man g a ▁review ▁in ▁a ▁ma g a z ine , ▁I ▁thought ▁it ▁was ▁maybe ▁interesting , ▁but ▁when'

In [10]:
first(spacy(['The U.S. dollar 1.00.']))

(#5) ['The','U.S.','dollar','1.00','.']

In [11]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#276) ['xxbos','xxmaj','this','is','one','of','the','worst','anime','series','i','have','ever','seen','.','xxmaj','when','i','watched','the','manga','review','in','a','magazine',',','i','thought','it','was','maybe'...]


In [12]:
coll_repr(tkn('©   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

In [13]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#276) ['xxbos','xxmaj','this','is','one','of','the','worst','anime','series','i','have','ever','seen','.','xxmaj','when','i','watched','the','manga','review','in','a','magazine',',','i','thought','it','was','maybe'...]


In [14]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#276) ['xxbos','xxmaj','this','is','one','of','the','worst','anime','series'...]

In [15]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2200) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','of','to','is','in','it','i'...]"

In [16]:
nums = num(toks)[:20]; nums

TensorText([  2,   8,  21,  16,  44,  14,   9, 160,   0, 249,  19,  38, 153,
            135,  10,   8,  71,  19, 394,   9])

In [17]:
#hide_input
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
df     

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
1,movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
2,first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
3,how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
4,of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
5,will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [18]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
df     

,0,1,2,3,4
0,xxbos,xxmaj,in,this,chapter
1,movie,reviews,we,studied,in
2,first,we,will,look,at
3,how,to,customize,it,.
4,of,the,preprocessor,used,in
5,will,study,how,we,build


In [19]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
df

,0,1,2,3,4
0,",",we,will,go,back
1,chapter,1,and,dig,deeper
2,the,processing,steps,necessary,to
3,xxmaj,by,doing,this,","
4,the,data,block,xxup,api
5,a,language,model,and,train


In [20]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
df

,0,1,2,3,4
0,over,the,example,of,classifying
1,under,the,surface,.,xxmaj
2,convert,text,into,numbers,and
3,we,'ll,have,another,example
4,.,\n,xxmaj,then,we
5,it,for,a,while,.


In [21]:
nums200 = toks200.map(num)


In [22]:
dl = LMDataLoader(nums200)

In [23]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [24]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj this is one of the worst xxunk series i have ever seen . xxmaj when i watched the'

In [25]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj this is one of the worst xxunk series i have ever seen . xxmaj when i watched the xxunk'

In [27]:
#get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

#dls_lm = DataBlock(
  #  blocks=TextBlock.from_folder(path, is_lm=True),
   # get_items=get_imdb, splitter=RandomSplitter(0.1)
#).dataloaders(path, path=path, bs=128, seq_len=80)

In [28]:
#dls_lm.show_batch(max_n=2)

In [30]:
#learn = language_model_learner(
 #   dls_lm, AWD_LSTM, drop_mult=0.3, 
  #  metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
#learn.fit_one_cycle(1, 2e-2)